# Titanic - Kaggle Competition: A trial run w Bayes

## Overview
This is an initial run through a Kaggle competition using a well known data set - passenger information from the Titanic and who survived.

The objective is to use machine learning to predict who would survive as a way of getting comfortable with how to submit entries to Kaggle.

### Data Dictionary

Variable	Definition	Key

survival	Survival	0 = No, 1 = Yes 

pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd

sex	Sex	
Age	Age in years	
sibsp	# of siblings / spouses aboard the Titanic	
parch	# of parents / children aboard the Titanic	
ticket	Ticket number	
fare	Passenger fare	
cabin	Cabin number	
embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton


### Variable Notes

pclass: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...
Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.



## Load and prep the data

In [ ]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
plt.style.use('ggplot')
%matplotlib inline

# signoid function, set as global lambda expression for use throughout
sig = lambda x: 1./(1+np.exp(-x))

In [ ]:
# load the data
train= pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.tail(3)

In [ ]:
test.head()

In [ ]:
# Save 'Survived' column into Y_train
Y_train = train['Survived']

In [ ]:
# merge data sets for transformations, now that survived stripped out
data = train.append(test)
data.reset_index(inplace=True)
data.drop('index',inplace=True,axis=1)

### Take a quick look at what we have
The first thing we will do is take a look at the data types and how much data is missing (null data) from each column.

In [ ]:
data.info()

In [ ]:
# looking at the signficant missing data (age and cabin),
# an approach to impute the values will need to be explored.

data.isnull().sum()

In [ ]:
# Cabin data is missing for 77% of the data
# leave this off the table for now, as no real measure to impute any useful information
# possibly use other data sources to fill in gaps
1014/1309.

## Feature Engineering: Break down names by title

In [ ]:
# dictionary to classify titles into major categories
titles = {  'Capt':        'Professional',
            'Col':         'Professional',
            'Major':       'Professional',
            'Jonkheer':    'Royal',
            'Don':         'Royal',
            'Sir' :        'Royal',
            'Dr':          'Professional',
            'Rev':         'Professional',
            'the Countess':'Royal',
            'Dona':        'Royal',
            'Mme':         'Mrs',
            'Mlle':        'Miss',
            'Ms':          'Mrs',
            'Mr' :         'Mr',
            'Mrs' :        'Mrs',
            'Miss' :       'Miss',
            'Master' :     'Master',
            'Lady' :       'Royal'
            }

# Title is between first comman and next period (e.g. , Mr. )
data['Title'] = data['Name'].map(lambda name:name.split(',')[1]\
                                 .split('.')[0].strip()).map(titles)
    

In [ ]:
data['Title'].unique()

In [ ]:
# look at median age by gender, class and title
ages = data.groupby(['Sex','Pclass','Title'])
ages.median()

In [ ]:
# impute missing ages by groupong by gender, class and title
data['Age'] = data.groupby(['Sex','Pclass','Title'])['Age'].transform(lambda x:
                                                                x.fillna(x.median()))


In [ ]:
# with Age imputed, leave cabin as is for now
data.isnull().sum()

## Feature Engineering: Categorize by Family Size

In [ ]:
data['Family'] = data['Parch'] + data['SibSp'] + 1

In [ ]:
# numbers don't sum completey because the passenger list is not complete
data['Family'].value_counts()

In [ ]:
data['Family_cat'] = data['Family'].map(lambda s :
                                    'alone' if s == 1 else 'small' if s < 5 else 'large')

In [ ]:
data['Family_cat'].value_counts()

## A look at major categories survival rate

In [ ]:
fig, ax2 = plt.subplots(3,2, figsize=(16,10))
category = 'Survival by Gender, Age, Class'

# set bin sizes to b the same
binBoundaries = np.linspace(0,80,9)

def details(j):
    if j == 0:
        pclass = 'First'
    elif j == 1:
        pclass = 'Second'
    else:
        pclass = 'Third'
    # sets titles for each subplot
    ax2[j][0].set_title('{0} Class # Survivors'.format(pclass))
    ax2[j][1].set_title('{0} Class # Deaths'.format(pclass)) 
    # sets common scale on all charts
    ax2[j][0].set_ylim(0,200)
    ax2[j][1].set_ylim(0,200)
    # turn on legends
    ax2[i][0].legend()
    ax2[j][1].legend()

for i in range(0,3):
    # survivors
    ax2[i][0].hist([data[(data['Survived']==1)&(data['Pclass']==i+1)&
                            (data['Sex']=='male')]['Age'], data[(data['Survived']==1)&
                            (data['Pclass']==i+1)&(data['Sex']=='female')]['Age']],
                             stacked=True, color = ['g','b'], bins = binBoundaries,
                             label = ['Male','Female'], alpha = 0.6)   
    # deaths
    ax2[i][1].hist([data[(data['Survived']==0)&(data['Pclass']==i+1)&
                            (data['Sex']=='male')]['Age'], data[(data['Survived']==0)&
                            (data['Pclass']==i+1)&(data['Sex']=='female')]['Age']],
                             stacked=True, color = ['g','b'], bins = binBoundaries,
                             label = ['Male','Female'], alpha = 0.6)
    # set graph lables/titles, etc
    details(i)

# font size for main title
fig.suptitle(category, fontsize=30);

## Feature Engineering: Creating Age Brackets/Buckets

In [ ]:
data['Age_cat'] = data['Age'].map(lambda a: '0-5' if a < 6 else '6-15' if a <16 else '16-25'
                                    if a < 26 else '26-35' if a <36 else '36-45' if a < 46
                                    else '46+')

In [ ]:
data['Age_cat'].value_counts()

### Ticket Data
Let's take a look at the ticket data to see how we may be able to use it.

In [ ]:
data['Ticket'].value_counts()

In [ ]:
data['Ticket_num'] =\
    data['Ticket'].apply(lambda x: pd.Series(x.split()[-1]))
data['tick_len'] = data['Ticket_num'].apply(lambda x: pd.Series(len(x)))

In [ ]:
data.describe()

## Use Seaborn pairplot to get a quick view of relationships

In [ ]:
sns.pairplot(data=data[['Fare','Survived','Age','Sex','Family','Pclass','tick_len']],
             hue='Survived');

In [ ]:
# 15 zero fares, is this correct, or shoudl they be imputed?
# looking at Titanic data, some passengers did not pay a fair, either comped or company
# sponsored, so difficult to call how to treat this
data[data['Fare']==0.0]['Fare'].value_counts()

In [ ]:
# Look at distribution of ticket prices

plt.hist([data[data['Pclass']==1]['Fare'],
         data[data['Pclass']==2]['Fare'],
         data[data['Pclass']==3]['Fare']],
         stacked=True, color = ['g','b','y'], bins = 30,
         label = ['1','2','3'], alpha = 0.6)

plt.legend();

## Feature Engineering: Ticket Price Buckets

In [ ]:
data['Fare_cat'] = data['Fare'].map(lambda f: '0-20' if f < 21 else '21-40' if f <41
                                      else '41-60' if f < 61 else '61-80' if f <81
                                      else '81-100' if f < 101 else '101+')

In [ ]:
data.head()

## Columns to remove in prep for Classificaiton Model

* Passenger ID
* Name - we have extracted title
* Age - we have put Age into categores 'Age_cat'
* Ticket - have created 'tick_len'
* Fare - have created 'Fare_cat'
* Cabin - too much missing data
* Family - have put into larger buckets in 'Family_cat'
* Ticket_num - used to create 'tick_len'

In [ ]:
data.tail()

In [ ]:
train.head()

In [ ]:
# change numbers to strings before get_dummies

to_num_str = ['Pclass','SibSp','Parch','tick_len']

for s in to_num_str:
    data[s] = data[s].apply(str)

In [ ]:
Y_train = train['Survived']

In [ ]:
len(Y_train)

In [ ]:
data = data.drop(['PassengerId','Name','Age','Ticket','Fare','Cabin','Family','Ticket_num',
                  'Survived'],1)

In [ ]:
data.head()

In [ ]:
data = pd.get_dummies(data)

In [ ]:
data.info()

In [ ]:
# drop one column from each set of dummy variables
data = data.drop(['Pclass_1','Sex_male','SibSp_8','Parch_6','Embarked_S',
                        'Title_Royal','Family_cat_large','Age_cat_46+','tick_len_1',
                        'Fare_cat_101+'],1)

In [ ]:
# Slice data back into train/test
# Y_train data already removed from data df

X_train = data.ix[0:890]
X_test = data.ix[891:]

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
# Set up data with survived as first column
train = X_train
train.insert(0, 'Survived', Y_train)


In [ ]:
train.head()

In [ ]:
train_list = train.values.tolist()

Add Survived as first column - to correspond to index[0]



# Bayesian Approach

## The Distribution Model

The model contains 41 parameters as a type of logistic regression, so the model is set up as follows:

### Likelihood

$$g(x) =\omega_0 + \omega_1param_1 + \omega_2param_2 + ... \omega_{41} param_{41}$$ 

$$\sigma(x) = \frac{1}{1+e^{-x}}$$

$$p = \sigma(g) = \frac{1}{1+e^{-g}}$$

So the pdf of the likelihood becomes: 
$$log(\sigma(g)) = -log({1+e^{-g}})$$

### Prior

For the Prior $\pi$ we chose the following over $\omega_0,\omega_1, \omega_2, \omega_3, ..., \omega_{40}$:

$$ \pi(\omega) = \frac{1}{2 \pi} e^{-\frac{1}{2}\omega_0^2}e^{-\frac{1}{2}\omega_1^2} e^{-\frac{1}{2}\omega_2^2} ... e^{-\frac{1}{2}\omega_{41}^2} $$

the log prior is:

$$ =  -\frac{1}{2}(\omega_0^2 + \omega_1^2+ \omega_2^2 +...+ \omega_{41}^2) $$


### Constraints

1. if status == 1 then return P
2. if status == 0 then return 1-p


In [ ]:
sigmoid = lambda g: 1./(1+np.exp(-g))

def log_predictive(w,dd):
    g = w[0] + np.sum([w[i]*dd[i] for i in range(1,40)])
    if dd[0] == 1:
        return np.log(sigmoid(g))
    else:
        return np.log(1 - sigmoid(g))

def log_prior(w):
    return -0.5 * np.sum([i**2 for i in w]) 

def lnprob(w):
    return log_prior(w) + np.sum([log_predictive(w,d) for d in train_list])

In [ ]:
import emcee
import corner


In [ ]:
import timeit
start_time = timeit.default_timer() # to time how long code takes to run


# Run MCMC 

steps = 1000 # number of steps

nwalkers = 100
ndim = 41 # reduced by two variables (remove 1 each from sets of dummy variables)
p0 = np.random.rand(nwalkers*ndim).reshape((nwalkers,ndim))
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, threads = 75)
pos, prob, state = sampler.run_mcmc(p0, 100)
sampler.reset()
pos, prob, state = sampler.run_mcmc(pos, steps)
samples = sampler.flatchain

elapsed = timeit.default_timer() - start_time 

print('Elapsed time for MCMCM run: {0}'.format(elapsed))


In [ ]:
fig = corner.corner(samples)

In [ ]:

MC_samples = pd.DataFrame(samples)
MC_samples.to_csv('MC_samples.csv')

# Use LC project to set how to run test set


# Results of Test Set: ...